In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
from ipywidgets import interact
import scipy.stats as stats
import pandas as pd
import pymc as pm ## If this line gives you an error, uncomment and execute the following cell

Populating the interactive namespace from numpy and matplotlib


In [7]:
path_to_data = 'https://raw.githubusercontent.com/ccontrer/MATH509-Winter2025-JupyterNotebooks/main/Data/'
data_all = pd.read_csv(path_to_data + "Howell1.csv", sep=";", header=0)
data_all

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0
...,...,...,...,...
539,145.415,31.127751,17.0,1
540,162.560,52.163080,31.0,1
541,156.210,54.062497,21.0,0
542,71.120,8.051258,0.0,1


# Q1
Please use the code from the Week 3 notebook and modify it as necessary to answer this question.

Suppose a colleague of yours, who works on allometry, glances at the practice problems just above. Your colleague exclaims, “That’s silly. Everyone knows that it’s only the logarithm of body weight that scales with height!” Let’s take your colleague’s advice and see what happens.


# A
Model the relationship between height (cm) and the natural logarithm of weight (log-kg). Use the **entire** Howell1 data frame, all 544 rows, adults and non-adults.

\begin{align*}
    h_{i} &\sim \text{Normal}(\mu_{i}, \sigma) \\
    \mu_{i} &= \alpha + e^{\log{\beta}} \log(w_i) \\
    \alpha & \sim \\
    \log(\beta) & \sim \\
    \sigma &\sim
\end{align*}

# B
To check the prior, plot 100 sample curves from prior predictive simulations (see Week 4 notebook for examples). Is the prior reasonable? Do you think it should be changed? Why or why not?

# C
Approximate the posterior distribution using the MCMC approximation (using Pymc3).

# D
Create a plot with the following information (each is a function of weight)
  1. a scatter plot of the data (height vs weight)
  2. predicted mean height
  3. the 97% interval for the mean height (use `fillbetween`)
  4. the 97% interval for posterior predicted heights (use `fillbetween`)
  
Be sure to label the $x$ and $y$ axis. Make it look nice.

In [20]:
#Set random seed to replicate results
RANDOM_SEED = 8927
random.seed(RANDOM_SEED)

pm.Normal
with pm.Model() as model_4_3:
    sigma = pm.Uniform("sigma", 0, data_all['height'].std()) ## uniform prior for sigma
    alpha = pm.Normal("alpha", mu = data_all['height'].mean(), sigma=data_all['height'].std()) ## normal prior for alpha
    logbeta = pm.Normal("logbeta", mu=0, sigma=1) ## normal prior for logbeta
    mu = alpha + exp(logbeta)*(data_all['weight'] - data_all['weight'].mean()) ## linear model (hidden variable)
    height = pm.Normal("height", mu=mu, sigma=sigma, observed=data_all['height']) ## likelihood
    trace_4_3 = pm.sample(1000, tune=1000) ## these are the MCMC samples!

trace_4_3_df = trace_4_3.posterior.to_dataframe()
trace_4_3_df

Output()

Output()

alpha   logbeta     sigma
chain draw                                
0     0     138.040296  0.548198  9.458036
      1     138.744089  0.580482  9.422047
      2     137.717245  0.541637  9.710080
      3     139.038609  0.590779  9.348116
      4     137.878042  0.538162  9.100213
...                ...       ...       ...
1     995   138.181118  0.571753  9.174830
      996   138.405656  0.564138  9.566197
      997   137.531265  0.577282  9.393286
      998   138.966766  0.576465  9.218868
      999   137.721011  0.551067  9.796025

[2000 rows x 3 columns]